<a href="https://colab.research.google.com/github/shm0007/fine-tuned-GPT2/blob/main/GPT2_Madam_Christie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine Tuned GPT2
## 02/22/22
Fine tuned version of GPT-2 with a text corpus consists of Agatha Christies novel.  


In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
#download the corpus zip file and unzip
!wget --no-check-certificate https://github.com/shm0007/corpus-crime/raw/main/corpus.zip
!unzip corpus.zip

In [ ]:
!rm -r corpus

In [ ]:
import os
from sklearn.model_selection import train_test_split
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

#get the file names and path from the corpus
current_path = os.getcwd()
os.listdir(current_path + "/corpus" )
filepaths = os.listdir(current_path + "/corpus" )

data = []

#str to store all the file names. Used it late to run with wc and getting token count
filenames  = ""

#loop through each file, preprocess and add to data
for fl in filepaths:
  f = open("corpus/" + fl)
  filenames += (" corpus/" +fl)
  file_text = f.read()
  f.close()

  #replace unnesessary characters with whitespace
  file_text = file_text.replace("\n", " ")
  file_text = file_text.replace("=", " ")
  file_text = file_text.replace("“", " ")
  file_text = file_text.replace("”", " ")
  sentences = sent_tokenize(file_text)
  for sentence in sentences:
    if(len(sentence) > 200): # remove sentences that are too long
      continue
    data.append(sentence)

train, test = train_test_split(data,test_size=0.15) #85 percent train, 15% validation
print(len(data))
print(len(train))
print(len(test))

#modified the function according to purpose, just kept the name same as the reference tutorial
def build_text_files(dt, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for line in dt:
        data += line
        #add a new line to the end of each sentence. Removing it increases training and validation loss.
        data += "\n"
    f.write(data)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

#print(filenames)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


The_Secret_of_Chimneys.txt
And_Then_There_Were_None.txt
The_Hunters_Lodge_Case.txt
The_Big_Four.txt
The_Man_in_the_Brown_Suit.txt
The_Secret_Adversary.txt
The_Mysterious_Affair_at_Styles.txt
The_Plymouth_Express_Affair.txt
The_Mystery_of_the_Blue_Train.txt
The_Murder_on_the_Links.txt
The_murder_of_Roger_Ackroyd.txt
Poirot_Investigates.txt
The_Missing_Will.txt
Large0
63258
53769
9489
 corpus/The_Secret_of_Chimneys.txt corpus/And_Then_There_Were_None.txt corpus/The_Hunters_Lodge_Case.txt corpus/The_Big_Four.txt corpus/The_Man_in_the_Brown_Suit.txt corpus/The_Secret_Adversary.txt corpus/The_Mysterious_Affair_at_Styles.txt corpus/The_Plymouth_Express_Affair.txt corpus/The_Mystery_of_the_Blue_Train.txt corpus/The_Murder_on_the_Links.txt corpus/The_murder_of_Roger_Ackroyd.txt corpus/Poirot_Investigates.txt corpus/The_Missing_Will.txt


In [ ]:
#read the word count for each of the book after pre processing
!wc corpus/The_Murder_on_the_Links.txt corpus/The_Hunters_Lodge_Case.txt corpus/The_Missing_Will.txt corpus/The_Mysterious_Affair_at_Styles.txt corpus/The_Secret_of_Chimneys.txt corpus/And_Then_There_Were_None.txt corpus/The_murder_of_Roger_Ackroyd.txt corpus/Poirot_Investigates.txt corpus/The_Big_Four.txt corpus/The_Mystery_of_the_Blue_Train.txt corpus/The_Plymouth_Express_Affair.txt corpus/The_Secret_Adversary.txt corpus/The_Man_in_the_Brown_Suit.txt

   9217   64470  383665 corpus/The_Murder_on_the_Links.txt
    574    4352   25602 corpus/The_Hunters_Lodge_Case.txt
    409    3257   19004 corpus/The_Missing_Will.txt
   8488   56456  341202 corpus/The_Mysterious_Affair_at_Styles.txt
  11399   74431  455894 corpus/The_Secret_of_Chimneys.txt
   7988   52607  320398 corpus/And_Then_There_Were_None.txt
  10355   69485  416920 corpus/The_murder_of_Roger_Ackroyd.txt
   7494   52494  313466 corpus/Poirot_Investigates.txt
   4227   55230  319360 corpus/The_Big_Four.txt
   7214   71222  414922 corpus/The_Mystery_of_the_Blue_Train.txt
    719    4858   29493 corpus/The_Plymouth_Express_Affair.txt
  10855   75138  452531 corpus/The_Secret_Adversary.txt
  10317   75261  435752 corpus/The_Man_in_the_Brown_Suit.txt
  89256  659261 3928209 total


In [ ]:
!wc train_dataset.txt test_dataset.txt

  53786  536748 2991664 train_dataset.txt
   9492   94080  524372 test_dataset.txt
  63278  630828 3516036 total


In [ ]:
# This block is used as is in the tutorial. Found out that TextDataset is old approach,
# tried with other methods, but couldn't make it with other methods.

from transformers import AutoTokenizer
from transformers import GPT2Tokenizer, GPT2Model,DataCollatorWithPadding
from transformers import TextDataset,DataCollatorForLanguageModeling
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'



def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

In [ ]:
#For logging to huggingface hub for pushing the model
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
#for clearing the gpu cache
torch.cuda.empty_cache()

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead,GPT2Model,PretrainedConfig

import evaluate
import numpy as np


model = AutoModelWithLMHead.from_pretrained("gpt2")

#modified some Traning Argument from the tutorial for better performance/see result
training_args = TrainingArguments(
    "gpt2-finetuned-agatha-christie", #The model name
    push_to_hub=True, #to push it directly while training
    save_strategy="steps", # it is saving after 50 steps
    evaluation_strategy = "steps", #after 'eval_steps" steps, it will evaluate
    num_train_epochs=6, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 50, # Number of update steps between two evaluations.
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    logging_steps=50
)


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6216
  Num Epochs = 6
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1170
  Number of trainable parameters = 124439808


Step,Training Loss,Validation Loss
50,4.282400,3.876366
100,3.882400,3.593053
150,3.633600,3.437750
200,3.505600,3.344468
250,3.403800,3.288068
300,3.350200,3.250618
350,3.313500,3.222385
400,3.283900,3.202802
450,3.219300,3.181643
500,3.206600,3.165961


***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1097
  Batch size = 64
Saving model checkpoint to gpt2-finetuned-agatha-christie/checkpoint-500
Configuration saved in gpt2-finetuned-agatha-christie/checkpoint-500/config.json
Configuration saved in gpt2-finetuned-agatha-christie/checkpoint-500/generation_config.json
Model weights saved in gpt2-finetuned-agatha

TrainOutput(global_step=1170, training_loss=3.2522835560334036, metrics={'train_runtime': 1597.1331, 'train_samples_per_second': 23.352, 'train_steps_per_second': 0.733, 'total_flos': 2436286906368000.0, 'train_loss': 3.2522835560334036, 'epoch': 6.0})

In [ ]:
trainer.push_to_hub()

#Examples
Used my model shm0007/gpt2-finetuned-agatha-christie which is pushed to huggingface in pipeline() function to generate text. One example was also taken using the model card interface api

In [ ]:
from transformers import pipeline
agatha = pipeline(task="text-generation",model="shm0007/gpt2-finetuned-agatha-christie")

In [ ]:
agatha("he stole and",max_length=110)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "he stole and looked at the ceiling.\nHe turned and went upstairs to his work.\nWhat was the name of the man you had met in South Africa last night?\nYou heard, then, that the maid’s voice was calling for help?\nI will never understand!\nWhat about all this mysterious woman?\nOh, I mean.\nYou don't believe me, Mr. Poirot,  said Virginia.\nBut of course he was a great soldier—and even that he was guilty’s own"}]

In [ ]:
agatha("Police came",max_length=110)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Police came forward and took the keys of the car.\nI knew,  I said, surprised,  that you had.\nThe doctor stood up and bowed.\nBut you and the Comte may have got away from those things, sir.\nHe had arrived at the house in a minute, and the two of them both stood to breakfast.\nI was the second person at work here.\nI’m sorry, I’ll see you later.\nAnd, you see, to know anything is not to be'}]

In [ ]:
agatha("President is shot on the head. he is murdered,said",max_length=110)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'President is shot on the head. he is murdered,said Colonel Race.\nNo one likes to be taken down by the wind.\nHe is the most efficient, you should guess, _mon ami._    Of course, Mrs. Blair?\nBut I know the truth.\nIt was in a book I was working on that day which I had forgotten.\nThis is your very first visitor to the island.\nWhat?\nWhy do you make so many such claims of your own?\nI’m so'}]

In [ ]:
agatha("How Poirot was killed?",max_length=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'How Poirot was killed?\nIt seemed that he was in all he went wrong.\nHe had made a brief but amusing impression to myself on many occasions.\nWe’ll hear about that.\nThen Tommy’s hand rested upon the edge of the receiver’s pocket-knife.\nThey were on to a theory that he was a Russian.\nBut the good Mr. Ackroyd would have preferred it if he had.\nNo.\nThere was'}]